In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q tensorflow==2.1
!pip install datasets transformers[sentencepiece] sacrebleu

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 87.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.3 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement tensorflow==2.1 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.4.4, 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0)
ERROR: No matching distribution found for tensorflow==2.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/p

In [2]:
import transformers

print(transformers.__version__)

4.27.0.dev0


In [3]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
# Importing libraries.
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
!gdown 1qvzJkNo7Dw_9bFSOts52xKkv1KzixLeA
!gdown 1dd-HKVeE6IbLVPKFBDTseIFG6aaCwC5m

Downloading...
From: https://drive.google.com/uc?id=1qvzJkNo7Dw_9bFSOts52xKkv1KzixLeA
To: /content/output_command.txt
100% 47.1k/47.1k [00:00<00:00, 55.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1dd-HKVeE6IbLVPKFBDTseIFG6aaCwC5m
To: /content/output_description.txt
100% 202k/202k [00:00<00:00, 103MB/s]


In [6]:
def load_csv(input, output):
  # Input: csv_file_name
  # Output: pandas data frame
  input_df = pd.read_csv(input, sep="\n", header=None, names=['input'])
  output_df = pd.read_csv(output, sep="\n", header=None, names=['output'])
  return input_df, output_df

In [10]:
input, output = load_csv('output_description.txt', 'output_command.txt')

overview = pd.concat([input, output], axis=1)
overview = overview.drop_duplicates()
overview

,input,output
0,Lists all files and directories in the present...,ls -l
1,Lists all files and directories in the present...,ls -la
2,Lists all files and directories in the present...,ls -lah
3,Lists all files and directories in the present...,ls -lahi
4,Lists all files and directories in the present...,ls -lahiF
...,...,...
3699,Display the system's current user.,whoami
3700,Display the system's current user's home direc...,echo $HOME
3701,Display the system's current user's shell.,echo $SHELL
3702,Display the system's current user's path.,echo $PATH


**Data Split**
*   Train: 10085 entries
*   Test: 1261 entries
*   Validation: 1261 entries

In [11]:
x_train, x_test, y_train, y_test = train_test_split(input, output, train_size=0.8, test_size=0.2, shuffle=True, random_state=0)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, train_size=0.5, test_size=0.5, shuffle=True, random_state=0)

train = pd.concat([x_train, y_train], axis=1)
test = pd.concat([x_test, y_test], axis=1)
valid = pd.concat([x_val, y_val], axis=1)

In [12]:
from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_pandas(train)
train_dataset = train_dataset.remove_columns(["__index_level_0__"])

test_dataset = Dataset.from_pandas(test)
test_dataset = test_dataset.remove_columns(["__index_level_0__"])

valid_dataset = Dataset.from_pandas(valid)
valid_dataset = valid_dataset.remove_columns(["__index_level_0__"])

raw_datasets = DatasetDict({"train":train_dataset, "test":test_dataset, "valid":valid_dataset})
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 2963
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 371
    })
    valid: Dataset({
        features: ['input', 'output'],
        num_rows: 370
    })
})

In [13]:
raw_datasets['test'][1]

{'input': 'Show the current processes running on the system.', 'output': 'ps'}

In [14]:
max_input_length = 128
max_target_length = 128

def preprocess_function(examples):
    inputs = [ex for ex in examples['input']]
    targets = [ex for ex in examples['output']]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [15]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_checkpoint = 't5-small'
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [16]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [17]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3586: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [18]:
from datasets import load_dataset, load_metric

metric = load_metric("sacrebleu")

<ipython-input-18-351bb915b7a7>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [19]:
batch_size = 5
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-transferLearning-NLP2BASH",
    evaluation_strategy = "epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [20]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [21]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [22]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/alexsha/t5-small-transferLearning-NLP2BASH into local empty directory.


Download file pytorch_model.bin:   0%|          | 17.4k/231M [00:00<?, ?B/s]

Download file runs/Feb02_23-43-46_d8b4e18acb50/1675381432.6846328/events.out.tfevents.1675381432.d8b4e18acb50.…

Download file runs/Feb02_23-43-46_d8b4e18acb50/events.out.tfevents.1675381432.d8b4e18acb50.417.4: 100%|#######…

Download file runs/Feb02_23-50-34_d8b4e18acb50/events.out.tfevents.1675381838.d8b4e18acb50.417.6: 100%|#######…

...

Download file training_args.bin: 100%|##########| 3.62k/3.62k [00:00<?, ?B/s]

Clean file training_args.bin:  28%|##7       | 1.00k/3.62k [00:00<?, ?B/s]

Clean file spiece.model:   0%|          | 1.00k/773k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/231M [00:00<?, ?B/s]

Using cuda_amp half precision backend


In [23]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: output, input. If output, input are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:346: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2963
  Num Epochs = 5
  Instantaneous batch size per device = 5
  Total train batch size (w. parallel, distributed & accumulation) = 5
  Gradient Accumulation steps = 1
  Total optimization steps = 2965
  Number of trainable parameters = 60506624
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using 

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,2.186600,0.902376,45.049900,5.383800
2,1.054600,0.621924,61.154500,5.775700
3,0.806200,0.537791,66.294400,5.524300
4,0.674200,0.490289,68.023900,5.613500
5,0.622600,0.476758,68.251300,5.645900


Saving model checkpoint to t5-small-transferLearning-NLP2BASH/checkpoint-500
Configuration saved in t5-small-transferLearning-NLP2BASH/checkpoint-500/config.json
Configuration saved in t5-small-transferLearning-NLP2BASH/checkpoint-500/generation_config.json
Model weights saved in t5-small-transferLearning-NLP2BASH/checkpoint-500/pytorch_model.bin
tokenizer config file saved in t5-small-transferLearning-NLP2BASH/checkpoint-500/tokenizer_config.json
Special tokens file saved in t5-small-transferLearning-NLP2BASH/checkpoint-500/special_tokens_map.json
Copy vocab file to t5-small-transferLearning-NLP2BASH/checkpoint-500/spiece.model
tokenizer config file saved in t5-small-transferLearning-NLP2BASH/tokenizer_config.json
Special tokens file saved in t5-small-transferLearning-NLP2BASH/special_tokens_map.json
Copy vocab file to t5-small-transferLearning-NLP2BASH/spiece.model
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` 

TrainOutput(global_step=2965, training_loss=0.9866608180645947, metrics={'train_runtime': 477.3438, 'train_samples_per_second': 31.036, 'train_steps_per_second': 6.211, 'total_flos': 82030348664832.0, 'train_loss': 0.9866608180645947, 'epoch': 5.0})

In [24]:
trainer.push_to_hub()

Saving model checkpoint to t5-small-transferLearning-NLP2BASH
Configuration saved in t5-small-transferLearning-NLP2BASH/config.json
Configuration saved in t5-small-transferLearning-NLP2BASH/generation_config.json
Model weights saved in t5-small-transferLearning-NLP2BASH/pytorch_model.bin
tokenizer config file saved in t5-small-transferLearning-NLP2BASH/tokenizer_config.json
Special tokens file saved in t5-small-transferLearning-NLP2BASH/special_tokens_map.json
Copy vocab file to t5-small-transferLearning-NLP2BASH/spiece.model


Upload file pytorch_model.bin:   0%|          | 32.0k/231M [00:00<?, ?B/s]

Upload file runs/Feb07_21-41-04_c4fb07e5f473/events.out.tfevents.1675806134.c4fb07e5f473.611.0: 100%|#########…

remote: Scanning LFS files for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/alexsha/t5-small-transferLearning-NLP2BASH
   844c707..18bb59c  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/alexsha/t5-small-transferLearning-NLP2BASH
   844c707..18bb59c  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}, 'metrics': [{'name': 'Bleu', 'type': 'bleu', 'value': 68.2513}]}
To https://huggingface.co/alexsha/t5-small-transferLearning-NLP2BASH
   18bb59c..bda8411  main -> main

   18bb59c..bda8411  main -> main



'https://huggingface.co/alexsha/t5-small-transferLearning-NLP2BASH/commit/18bb59cfb8b7860fe380d275a114830ecae99dbe'